<a href="https://colab.research.google.com/github/shivavsrivastava/Algorithms/blob/main/Course4_W3_TravelingSalesmanProblem_Heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # Traveling Salesman Problem (TSP)

Use Heuristic to solve this NP-complete problem in polynomial time. Sacrifice accuracy to reach an approximate solution

In [ ]:
import numpy as np
import random
import urllib3
import math
import time
import copy
import sys
from itertools import combinations, permutations


import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def TravelingSalesmanProblem_Heuristic(graph):
    nodes_list = list(graph.nodes)
    not_visited = list(graph.nodes)
    print(not_visited)
    start_node = not_visited[0]
    print(start_node)
    x = start_node
    x_index = nodes_list.index(x)
    closest_neighbor = not_visited[-1]
    print(closest_neighbor)
    closest_neighbor_index = nodes_list.index(closest_neighbor)
    tsp_tour = 0
    not_visited.remove(start_node)
    while not_visited:
        neighbors = list(graph.neighbors(x))
        print("neighbors {} of {}".format(neighbors, x))
        closest_neighbor_dist = float('inf')
        for neighbor in neighbors:
            if neighbor in not_visited:
                dist = graph.edges[x, neighbor]['weight']
                print("dist {} of {}".format(dist, neighbor))
                if (dist < closest_neighbor_dist) or (dist==closest_neighbor_dist and nodes_list.index(neighbor)<closest_neighbor_index) :
                    closest_neighbor_dist = dist
                    closest_neighbor = neighbor
                    closest_neighbor_index = nodes_list.index(neighbor)
        print(closest_neighbor)
        tsp_tour += closest_neighbor_dist
        not_visited.remove(closest_neighbor)
        x = closest_neighbor
    tsp_tour += graph.edges[start_node, x]['weight']
    return tsp_tour



Tests : small graphs

In [ ]:
G = nx.Graph()
for i,j in zip(['a', 'b', 'c', 'd', 'e'],[(5, 5), (5, 10), (0, 5), (0, 10), (-5, 7.5)]):
    G.add_node(i, pos=j)
G.add_weighted_edges_from([('a', 'b', 1), ('b', 'd', 2), ('d', 'c', 3), ('e', 'a', 4), ('d', 'c', 5), ('c', 'e', 6)])
pos = nx.get_node_attributes(G, 'pos')
weights = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels = weights)
nx.draw_networkx_labels(G,pos=pos)
nx.draw(G, pos, with_labels = True)
TravelingSalesmanProblem_Heuristic(G)
# dynamic solution was 18

In [ ]:
G = nx.Graph()
for i,j in zip(['s', 'a', 'd', 'g', 'h', 'i', 't'],[(0, 5), (5, 10), (5, 5), (5, 0), (10, 0), (15, 0), (20, 5)]):
    G.add_node(i, pos=j)
G.add_weighted_edges_from([('s', 'a', 1), ('s', 'd', 1), ('s', 'g', 1), ('a', 't', 1), ('d', 't', 100), ('g', 'h', 1), ('h', 'i', 1), ('i', 't', 100), ('i', 'a', 1), ('d', 'a', 1)])
pos = nx.get_node_attributes(G, 'pos')
weights = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels = weights)
nx.draw_networkx_labels(G,pos=pos)
nx.draw(G, pos, with_labels = True)
TravelingSalesmanProblem_Heuristic(G)

## Programming Assignment



In this assignment we will revisit an old friend, the traveling salesman problem (TSP).  This week you will implement a heuristic for the TSP, rather than an exact algorithm, and as a result will be able to handle much larger problem sizes.  Here is a data file describing a TSP instance (original source:
http://www.math.uwaterloo.ca/tsp/world/bm33708.tsp
).

**nn.txt**


The first line indicates the number of cities. Each city is a point in the plane, and each subsequent line indicates the x- and y-coordinates of a single city.

The distance between two cities is defined as the Euclidean distance --- that is, two cities at locations
(
𝑥
,
𝑦
)
(x,y) and
(
𝑧
,
𝑤
)
(z,w) have distance
(
𝑥
−
𝑧
)
2
+
(
𝑦
−
𝑤
)
2
(x−z)
2
 +(y−w)
2

​
  between them.

You should implement the nearest neighbor heuristic:

Start the tour at the first city.

Repeatedly visit the closest city that the tour hasn't visited yet.  In case of a tie, go to the closest city with the lowest index.  For example, if both the third and fifth cities have the same distance from the first city (and are closer than any other city), then the tour should begin by going from the first city to the third city.

Once every city has been visited exactly once, return to the first city to complete the tour.

In the box below, enter the cost of the traveling salesman tour computed by the nearest neighbor heuristic for this instance, rounded down to the nearest integer.

[Hint: when constructing the tour, you might find it simpler to work with squared Euclidean distances (i.e., the formula above but without the square root) than Euclidean distances.  But don't forget to report the length of the tour in terms of standard Euclidean distance.]

In [ ]:
def testcase_tsp(url):
    http = urllib3.PoolManager()
    r1 = http.request('GET', url)
    IntegerMatrixStringJoin = r1.data.decode('utf8').split('\n')
    IntegerMatrixStringJoin.remove('')
    # first line is number of nodes
    n = int(IntegerMatrixStringJoin[0])
    IntegerMatrixStringJoin.remove(IntegerMatrixStringJoin[0])
    coords_cities = {}
    # This graph is really big as it has all the edges from one city to another
    for i in IntegerMatrixStringJoin:
        node_coord = i.split(' ')
        city = int(node_coord[0])
        x = float(node_coord[1])
        y = float(node_coord[2])
        coords_cities[city] = (x, y)

    # From here on I will tack the TSP tour logic
    not_visited = [_ for _ in range(1, n+1)]
    x = 1
    tsp_tour = 0
    not_visited.remove(x)
    while not_visited:
        min_dist_left = float('inf')
        min_dist_right = float('inf')
        closest_neighbor_left = 1
        closest_neighbor_right = n
        # Search left neighbor for minimum distance
        for neighbor in range(2, x):
            if neighbor in not_visited:
                euclidean_distance = ((coords_cities[x][0]-coords_cities[neighbor][0])**2 + \
                            (coords_cities[x][1]-coords_cities[neighbor][1])**2)**0.5
                x_dist = abs(coords_cities[x][0]-coords_cities[neighbor][0])
                if (euclidean_distance < min_dist_left) or (euclidean_distance==min_dist_left and neighbor < closest_neighbor_left):
                    min_dist_left = euclidean_distance
                    closest_neighbor_left = neighbor
                # if x_dist exceeds the min_dist then no need to look at neighbors anymore
                if x_dist > min_dist_left:
                    break
        # Search right neighbor for minimum distance
        for neighbor in range(x+1, n+1):
            if neighbor in not_visited:
                euclidean_distance = ((coords_cities[x][0]-coords_cities[neighbor][0])**2 + \
                            (coords_cities[x][1]-coords_cities[neighbor][1])**2)**0.5
                x_dist = abs(coords_cities[x][0]-coords_cities[neighbor][0])
                if (euclidean_distance < min_dist_right) or (euclidean_distance==min_dist_right and neighbor < closest_neighbor_right):
                    min_dist_right = euclidean_distance
                    closest_neighbor_right = neighbor
                # if x_dist exceeds the min_dist then no need to look at neighbors anymore
                if x_dist > min_dist_right:
                    break
        # Get the minimum distance of the left and right neighbors
        if(min_dist_left < min_dist_right):
            closest_neighbor = closest_neighbor_left
            tsp_tour += min_dist_left
        else:
            closest_neighbor = closest_neighbor_right
            tsp_tour += min_dist_right

        print("Current node = {}, tsp_tour value = {} for closest node {}".format(x, tsp_tour, closest_neighbor))
        x = closest_neighbor
        not_visited.remove(closest_neighbor)

    tsp_tour += ((coords_cities[x][0]-coords_cities[1][0])**2 + \
                (coords_cities[x][1]-coords_cities[1][1])**2)**0.5
    print(tsp_tour)



In [ ]:
testcase_tsp("https://d3c33hcgiwev3.cloudfront.net/_ae5a820392a02042f87e3b437876cf19_nn.txt?Expires=1716768000&Signature=JTPfpEbbZrINCcjcgeGCGiyiI50-56R2R~TJ79XJs93-tcPyxCNcfCqPmHVjVHmVekDEy7bpNnz6xdPAxdEZBEmoM3VqM~22h8NorHZmdLAUB0N7aI1zjLNLCQ24dDHlx1G2~AeUcRlfxZsDKF~0aS2C2w4R4VhoYPIpmAm1Cq8_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")


Answer is 1203406.501271 and I did not get there, even after running this brute force algorithm for more than 2 hours, need to find a better solution